In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import statsmodels.formula.api as smf
from IPython.display import Latex, Markdown
from ISLP import load_data
from sklearn.discriminant_analysis import (
    LinearDiscriminantAnalysis,
    QuadraticDiscriminantAnalysis,
)
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier

weekly = load_data("Weekly")
default = load_data("default")
sns.set_theme(style="dark")

# Problem 1

$$
\begin{split}
p(X)&=\frac{e^{\beta_0+\beta_1 X}}{1+e^{\beta_0+\beta_1 X}} \\
\frac{p(X)}{1-p(X)}&=\frac{\frac{e^{\beta_0+\beta_1 X}}{1+e^{\beta_0+\beta_1 X}}}{1-\frac{e^{\beta_0+\beta_1 X}}{1+e^{\beta_0+\beta_1 X}}} \\
&=\frac{\frac{e^{\beta_0+\beta_1 X}}{1+e^{\beta_0+\beta_1 X}}}{\frac{1+e^{\beta_0+\beta_1 X}}{1+e^{\beta_0+\beta_1 X}}-\frac{e^{\beta_0+\beta_1 X}}{1+e^{\beta_0+\beta_1 X}}} \\
&=\frac{\frac{e^{\beta_0+\beta_1 X}}{1+e^{\beta_0+\beta_1 X}}}{\frac{1+e^{\beta_0+\beta_1 X}}{1+e^{\beta_0+\beta_1 X}}-\frac{e^{\beta_0+\beta_1 X}}{1+e^{\beta_0+\beta_1 X}}} \\
&=\frac{\frac{e^{\beta_0+\beta_1 X}}{1+e^{\beta_0+\beta_1 X}}}{\frac{1+e^{\beta_0+\beta_1 X}}{1+e^{\beta_0+\beta_1 X}}-\frac{e^{\beta_0+\beta_1 X}}{1+e^{\beta_0+\beta_1 X}}} \\
&=\frac{\frac{e^{\beta_0+\beta_1 X}}{1+e^{\beta_0+\beta_1 X}}}{\frac{1}{1+e^{\beta_0+\beta_1 X}}} \\
&=e^{\beta_0+\beta_1 X}
\end{split}
$$

# Problem 2

## Problem 2.a

On average, $\frac{1}{10}$ of the observations will be used to make the predictions.

## Problem 2.b

On average, $\frac{1}{100}$ of the observations will be used to make the predictions.

## Problem 2.c

On average, $10^{-100}$ of the observations will be used to be used to make the predictions.

## Problem 2.d

As the number of features increase, fewer of the larger set will be used for inference, which will make the model seem relatively 'dumber' as it sees a very local pictures and never really 'understanding' a larger picture, therefore also causing really high variance and overfitting.

## Problem 2.e

$$
\begin{split}
s^{p}&=10^{-1} \\
p\ln s&=-\ln 10 \\
\ln s&=-p^{-1}\ln 10 \\
s&=10^{-p^{-1}}
\end{split}
$$

### p=1

$$
s=10^{-1}=0.1
$$

### p=2

$$
s=10^{-0.5}\approx .32
$$

### p=100

$$
s=10^{-.01}\approx 0.98
$$

# Problem 3

## Problem 3.a

$$
Y=\sigma\left(\hat{\beta}_0+\hat{\beta}_1 X_1+\hat{\beta}_2 X_2\right)=\sigma\left(-6+40\cdot .05+1\cdot 3.5\right)=\sigma(-0.5)\approx .377
$$

## Problem 3.b

$$
\begin{split}
0.5&=\sigma\left(\hat{\beta}_0+\hat{\beta}_1 X_1+\hat{\beta}_2 X_2\right)=\sigma\left(-6+0.05h+3.5\right) \\
0.5&=\sigma\left(0.05h-2.5\right)
h\approx 50
\end{split}
$$

# Problem 4

We would prefer the regression, because the $K=1$ KNN will have a $0\%$ error on the training data ( the NN of any data point in the training data set will just be itself ), which means that it had a $36\%$ error on the testing data. Scince the $36\%$ test error for KNN is more than the $30\%$ error on the linear regression, we would prefer the regression to classify new observations.

# Problem 5

## Problem 5.a

In [ ]:
# | warning: false
sns.pairplot(weekly, hue="Direction", diag_kws={"multiple": "stack"})

From looking at the graphs, we can see that Today is clustered by Direction and there is a correlation between Volume and year.

## Problem 5.b

In [ ]:
weekly_binary = weekly.assign(Direction=(weekly["Direction"] == "Up").astype(int))
model = smf.logit(
    "Direction ~ Volume + Lag1 + Lag2 + Lag3 + Lag4 + Lag5", data=weekly_binary
).fit(disp=False)
model.summary()

Lag2 is the only statistically significant correlation.

## Problem 5.c

In [ ]:
prediction = model.predict(
    weekly_binary[["Volume", "Lag1", "Lag2", "Lag3", "Lag4", "Lag5"]]
).round()
ConfusionMatrixDisplay.from_predictions(weekly_binary["Direction"], prediction)
plt.show()

We can see that the model is very hesitant to guess 0, so it has a very low recall,

## Problem 5.d

In [ ]:
weekly_training = weekly[weekly["Year"] <= 2008]
weekly_training_binary = weekly_training.assign(
    Direction=(weekly_training["Direction"] == "Up").astype(int)
)
weekly_testing = weekly[weekly["Year"] >= 2009]
weekly_testing_binary = weekly_testing.assign(
    Direction=(weekly_testing["Direction"] == "Up").astype(int)
)
model = smf.logit("Direction ~ Lag2", data=weekly_training_binary).fit(disp=False)
prediction = model.predict(weekly_testing_binary[["Lag2"]]).round()
ConfusionMatrixDisplay.from_predictions(weekly_testing_binary["Direction"], prediction)
plt.show()

## Probelm 5.e

In [ ]:
model = LinearDiscriminantAnalysis().fit(
    weekly_training_binary[["Lag2"]], weekly_training_binary["Direction"]
)
prediction = model.predict(weekly_testing_binary[["Lag2"]]).round()
ConfusionMatrixDisplay.from_predictions(weekly_testing_binary["Direction"], prediction)
plt.show()

## Problem 5.f

In [ ]:
model = QuadraticDiscriminantAnalysis().fit(
    weekly_training_binary[["Lag2"]], weekly_training_binary["Direction"]
)
prediction = model.predict(weekly_testing_binary[["Lag2"]]).round()
ConfusionMatrixDisplay.from_predictions(weekly_testing_binary["Direction"], prediction)
plt.show()

## Problem 5.g

In [ ]:
model = KNeighborsClassifier(n_neighbors=1).fit(
    weekly_training_binary[["Lag2"]], weekly_training_binary["Direction"]
)
prediction = model.predict(weekly_testing_binary[["Lag2"]]).round()
ConfusionMatrixDisplay.from_predictions(weekly_testing_binary["Direction"], prediction)
plt.show()

## Probem 5.h

In [ ]:
model = GaussianNB().fit(
    weekly_training_binary[["Lag2"]], weekly_training_binary["Direction"]
)
prediction = model.predict(weekly_testing_binary[["Lag2"]]).round()
ConfusionMatrixDisplay.from_predictions(weekly_testing_binary["Direction"], prediction)
plt.show()

# Problem 6

## Problem 6.a

Each bootstrap observation has a $\frac{1}{n}$ chance of being equal to the $j$th element of the original sample, so it has a $1-\frac{1}{n}=\frac{n-1}{n}$ chance of not being the $j$th observation.

## Problem 6.b

Every bootstrap observation is sampled identically, so it is also $\frac{n-1}{n}$

## Problem 6.c

Each bootstrap observation has a $1-\frac{1}{n}$ probability of not being the $j$th observation, so the chance that every one of $n$ bootsrap observations in a bootstrap sample of size $n$ is not the $j$th observation is $\left(1-\frac{1}{n}\right)^n$.

## Problem 6.d

$\left(1-\frac{1}{n}\right)^n=\left(1-\frac{1}{5}\right)^5\approx 0.33$

## Problem 6.e

$\left(1-\frac{1}{n}\right)^n=\left(1-\frac{1}{100}\right)^{100}\approx 0.37$

## Problem 6.f

$\left(1-\frac{1}{n}\right)^n=\left(1-\frac{1}{10000}\right)^{10000}\approx 0.37$

## Problem 6.g

In [ ]:
x = np.arange(1, 10000)
sns.lineplot((1 - 1 / x) ** x)
plt.show()

$\left(1-\frac{1}{n}\right)^n$ increases and asymptotes to $e^{-1}$.

## Python 6.h

In [ ]:
rng = np.random.default_rng(10)
store = np.empty(10000)
for i in range(10000):
    store[i] = np.sum(rng.choice(100, replace=True) == 4) > 0
np.mean(store)

In [ ]:
np.random.seed(10)
np.mean(np.random.randint(1, 100, 10000) == 4)

These are both very different from the estimate of $\left(1-\frac{1}{100}\right)^{100}=.37$; the python ones are significantly lower and the R is is significantly higher.

# Problem 7

## Probblem 7.a

In [ ]:
default_binary = default.assign(
    default=(default["default"] == "Yes").astype(int),
    student=(default["student"] == "Yes").astype(int),
)
model = smf.logit("default ~ income + balance", data=default_binary).fit(disp=False)
model.summary()

## Problem 7.b

In [ ]:
default_binary_train, default_binary_test = train_test_split(
    default_binary, test_size=0.5, random_state=42
)
model = smf.logit("default ~ income + balance", data=default_binary_train).fit(
    disp=False
)
prediction = model.predict(default_binary_test[["income", "balance"]]).round()
error = sum(prediction != default_binary_test["default"]) / len(
    default_binary_test["default"]
)

The error is `{python} error`

## Problem 7.c

In [ ]:
splits = [0.25, 0.5, 0.75]
errors = []
for split in splits:
    default_binary_train, default_binary_test = train_test_split(
        default_binary, test_size=split, random_state=42
    )
    model = smf.logit("default ~ income + balance", data=default_binary_train).fit(
        disp=False
    )
    prediction = model.predict(default_binary_test[["income", "balance"]]).round()
    error = sum(
        prediction
        != (default_binary_test["default"]) / len(default_binary_test["default"])
    )
    errors.append(error)
Latex(pd.DataFrame({"Training Split": splits, "Error": errors}).to_latex())

From these there seems to be a almost parabolic motion centered somewhere around $0.5$, such that increasing or decreasing the training split will derease performance on the testing data.

## Problem 7.d

In [ ]:
default_binary_train, default_binary_test = train_test_split(
    default_binary, test_size=0.5, random_state=42
)
model = smf.logit(
    "default ~ student + income + balance", data=default_binary_train
).fit(disp=False)
prediction = model.predict(
    default_binary_test[["student", "income", "balance"]]
).round()
error = sum(prediction != default_binary_test["default"]) / len(
    default_binary_test["default"]
)

The error is `{python} error`, which is an improvment over the version without student, which means that student is providing more information to the model rather than just providing a point of overfitting for the training.

# Problem 8